In [ ]:
#Extract the ZIP File
import zipfile
import os

zip_path = "/content/drive/MyDrive/DL_Exp/Exp1/archive.zip"
extract_path = "/content/mnist_data"

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

print("Extraction completed!")

#check the extracted files:
os.listdir(extract_path)

#Load MNIST Data Using idx2numpy
import idx2numpy
import numpy as np

image_path = "/content/mnist_data/t10k-images.idx3-ubyte"  # Update path if needed
label_path = "/content/mnist_data/t10k-labels.idx1-ubyte"

images = idx2numpy.convert_from_file(image_path)  # Shape: (10000, 28, 28)
labels = idx2numpy.convert_from_file(label_path)  # Shape: (10000,)

# Normalize pixel values to [0, 1]
images = images.reshape(images.shape[0], -1) / 255.0  # Flatten to (10000, 784)

print(f"Images Shape: {images.shape}")  # (10000, 784)
print(f"Labels Shape: {labels.shape}")  # (10000,)

# One-Hot Encode Labels
def one_hot_encode(labels, num_classes=10):
    one_hot = np.zeros((labels.size, num_classes))
    one_hot[np.arange(labels.size), labels] = 1
    return one_hot

labels = one_hot_encode(labels)
print(f"Labels Shape (One-Hot): {labels.shape}")  # (10000, 10)

#Initialize Neural Network Parameters
#We will build a simple 2-layer neural network:

#Input: 784 neurons (flattened 28×28 image)
#Hidden Layer: 128 neurons (ReLU activation)
#Output: 10 neurons (Softmax for classification)
np.random.seed(42)

# Initialize weights and biases
input_size = 784
hidden_size = 128
output_size = 10

W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

# Define Activation Functions
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))  # Prevent overflow
    return expZ / np.sum(expZ, axis=1, keepdims=True)

def relu_derivative(Z):
    return Z > 0

# Implement Forward & Backward Propagation
def forward_propagation(X):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def backward_propagation(X, Y, Z1, A1, Z2, A2, learning_rate):
    global W1, b1, W2, b2

    m = X.shape[0]  # Number of samples

    # Compute gradients
    dZ2 = A2 - Y
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    # Update parameters
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

# Train the Model
epochs = 10
batch_size = 64
learning_rate = 0.01

for epoch in range(epochs):
    total_loss = 0

    # Shuffle dataset
    indices = np.arange(images.shape[0])
    np.random.shuffle(indices)
    images, labels = images[indices], labels[indices]

    # Mini-batch training
    for i in range(0, images.shape[0], batch_size):
        X_batch = images[i:i+batch_size]
        Y_batch = labels[i:i+batch_size]

        Z1, A1, Z2, A2 = forward_propagation(X_batch)
        loss = -np.mean(Y_batch * np.log(A2 + 1e-8))  # Cross-entropy loss
        backward_propagation(X_batch, Y_batch, Z1, A1, Z2, A2, learning_rate)

        total_loss += loss

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

# Evaluate the Model
def predict(X):
    _, _, _, A2 = forward_propagation(X)
    return np.argmax(A2, axis=1)

predictions = predict(images)
true_labels = np.argmax(labels, axis=1)

accuracy = np.mean(predictions == true_labels) * 100
print(f"Test Accuracy: {accuracy:.2f}%")




Extraction completed!
Images Shape: (10000, 784)
Labels Shape: (10000,)
Labels Shape (One-Hot): (10000, 10)
Epoch [1/10], Loss: 35.9943
Epoch [2/10], Loss: 35.2796
Epoch [3/10], Loss: 33.0666
Epoch [4/10], Loss: 28.2009
Epoch [5/10], Loss: 21.8599
Epoch [6/10], Loss: 16.9217
Epoch [7/10], Loss: 13.7153
Epoch [8/10], Loss: 11.6469
Epoch [9/10], Loss: 10.2780
Epoch [10/10], Loss: 9.2510
Test Accuracy: 85.80%
